In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [ ]:
persist_directory = os.environ.get("PERSIST_DIRECTORY", "/tmp")

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Prompts

In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
query = "Can you explain me how to use the pure market making strategy?"

In [ ]:
result = qa_chain({"query": query})

In [ ]:
result["result"]

In [ ]:
result["source_documents"]

# Testing ConverstionalQARetrieval

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    memory=memory
)

In [ ]:
result_conv = qa({"question": query})

In [ ]:
print(result_conv["answer"])

In [ ]:
result_conv_2 = qa.invoke({"question": "How the order refresh time is going to affect my strategy?"})

In [ ]:
result_conv_2["answer"]